## Импорты и загрузка данных

In [4]:
import numpy as np
import pandas as pd

import os

In [5]:
path = os.getcwd()

fish_df = pd.read_csv(path + r"\data\db1\ref\fish.csv", error_bad_lines=False, sep=';')
prod_designate_df = pd.read_csv(path + r"\data\db1\ref\prod_designate.csv", error_bad_lines=False, sep=';')
prod_type_df = pd.read_csv(path + r"\data\db1\ref\prod_type.csv", error_bad_lines=False, sep=';')
regime_df = pd.read_csv(path + r"\data\db1\ref\regime.csv", error_bad_lines=False, sep=';')
region_df = pd.read_csv(path + r"\data\db1\ref\region.csv", error_bad_lines=False, sep=';')

catch_df = pd.read_csv(path + r'\data\db1\catch.csv', error_bad_lines=False, sep=',')

# Подмена данных + нормализация
def spoofing(id):
    return fish_df[fish_df['id_fish'] == id]['fish'].values[0]
catch_df['fish'] = catch_df['id_fish'].apply(spoofing)
catch_df['catch_volume'] = catch_df['catch_volume']*1000

product_df = pd.read_csv(path + r'\data\db1\product.csv', error_bad_lines=False, sep=',')

ext1 = pd.read_csv(path + r"\data\db2\Ext.csv", error_bad_lines=False, sep=',')
ext2 = pd.read_csv(path + r"\data\db2\Ext2.csv", error_bad_lines=False, sep=',')

In [6]:
# Чистка больших данных от анонимных данных
ext1 = ext1.drop(ext1[ext1['id_fishery'] == -1].index)
ext1 = ext1.drop(columns=['Name_Plat', 'Product_period', 'Region_Plat', 'numPart'])

In [7]:
# Сводная таблица по всем поступлениям согласно IdFish и выбранного промежутка времени.
def calculatingSummOfFish(fish_name, date_start, date_end):
    print('Processing incoming values')
    # смотрим по всем выловам этой рыбы
    data = catch_df[catch_df['fish'] == fish_name]
    # уникальные номера кораблей ловящих только эту рыбу
    ships = list(set(data['id_ves'].to_list()))
    
    # создаю датафрейм с владельцами и выловом по судам    
    total_df = pd.DataFrame()
    
    total_ships = []
    total_owners = []
    total_catches = []
    
    # добавляем в общую кашу дату и документируем владельцев
    for ship in ships:
        ship_data = data[data['id_ves'] == ship]
        owners = list(set(ship_data['id_own'].to_list()))
        
        for owner in owners:
            ship_owner_data = ship_data[ship_data['id_own'] == owner]
            
            try:
                ship_owner_end_data = ship_owner_data[ship_owner_data['date'] < date_end]
                ship_owner_start_end_data = ship_owner_end_data[ship_owner_end_data['date'] > date_start]
                total_catch = ship_owner_start_end_data['catch_volume'].sum()
            except e:
                print(e)
            
            if total_catch > 0:
                total_catches.append(total_catch)
                total_ships.append(ship)
                total_owners.append(owner)
            
    total_df['total_catches'] = total_catches
    total_df['ship'] = total_ships
    total_df['owner'] = total_owners
    
    return total_df

In [8]:
# Вспомогательная функция вычисления для КОНКРЕТНЫХ ВЛАДЕЛЬЦЕВ ГРУПП СУДОВ, по FishID
def calculatingSummOfFishTransfering(fish_name, start, end):
    print('Processing transfering')
    total_owners = []
    total_transfers = []
    total_fishery_date = []
    total_vsd_date = []
    
    # Смотрю по каждому ID акты передачи    
    vsd_fish_id = ext2[ext2['fish'] == fish_name]['id_vsd']
    # Для каждого акта определяю владельца     
    for vsd in vsd_fish_id:
        owners = ext1[ext1['id_vsd'] == vsd]['id_own'].unique().tolist()
        # Проверяю является ли владелец связанным по двум базам  
        for owner in owners:
            if owner in market_owners:
                data = ext2[ext2['id_vsd'] == vsd]
                try:
                    total_owners.append(owner)
                    total_transfers.append(data['volume'].values[0])
                    total_fishery_date.append(data['date_vsd'].values[0][5:10])
                    total_vsd_date.append(ext1[ext1['id_vsd'] == vsd]['date_fishery'].values[0][5:10])
                except Exception() as e:
                    print(e)
                    
    df = pd.DataFrame()
    
    df['owner'] = total_owners
    df['volume'] = total_transfers
    df['fishery_date'] = total_fishery_date
    df['vsd_date'] = total_vsd_date
    
    return df

### Поиск аномалий

#### Осуществим с помощью вычленения ID рыбы и даты вылова (для простоты возьмем весь период, но можно брать любой промежуток)

In [9]:
# fish_id = input('Введите ID рыбы\n')

default_data = (catch_df['date'][0], catch_df['date'][len(catch_df) - 1])
start, end = default_data

#### Возьмем уникальный лист владельцев судов

In [10]:
catcher_owners = catch_df['id_own'].unique()
traider_owners = ext1['id_own'].unique()

market_owners = []
for owner in traider_owners:
    if owner in catcher_owners:
        market_owners.append(owner)  

In [11]:
print(len(catcher_owners), len(traider_owners), len(market_owners))

350 158 121


#### Как видим, db2 не полностью покрывает всех владельцев, следовательно, будем использовать для поиска только тех, которые задействованы в обоих БД

In [12]:
# Смотрим по общему поступлению владельца и общей передачи
def getAllTransfers(id, start, end):
    incoming = calculatingSummOfFish(id, start, end)
    outcoming = calculatingSummOfFishTransfering(id, start, end)
    # Важно учитывать неналичие в таблице вылова владельцев которые не передают ничего.
    inputs = incoming['owner'].unique().tolist()
    outputs = outcoming['owner'].unique().tolist()

    owners = []

    for item in inputs:
        if item in outputs:
            owners.append(item)

    total_catches = []
    total_transfered = []
    total_owners = []
    total_fish = []

    for owner in owners:
        
        catch = in_[in_['owner'] == owner]['total_catches'].sum()
        transfer = out[out['owner'] == owner]['volume'].sum()
        
        if catch > 0 or transfer > 0:
            print(len(inputs), len(outputs), len(owners))
            print(catch, transfer, owner)

            total_catches.append(catch)
            total_transfered.append(transfer)
            total_owners.append(owner)
            total_fish.append(id)

    df = pd.DataFrame()

    df['catches'] = total_catches
    df['transfered'] = total_transfered
    df['owner'] = total_owners
    df['fish'] = total_fish
    
    return df
    

#### Алгоритмическая обработка явных аномалий

##### Код ниже позволяет вычленять уникальные номера владельцев, при условии их наличия в обоих базах одновременно. Это дает возможность понять кто из них обманывает и насколько. 

In [126]:
fishes_id = fish_df['fish'].to_list()[1:]

total_df = getAllTransfers(fishes_id[0], start, end)

for fish in fishes_id[1:]:
    df = getAllTransfers(fish, start, end)
    total_df = total_df.append(df, ignore_index=True)
    if fishes_id.index(fish) % 10 == 0:
        total_df.to_csv(f'new_data/data{ fishes_id.index(fish) }.csv')
total_df

Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering


Processing transfering
Processing incoming values
Processing transfering
17 4 4
7869600.0 16794998 6462295
17 4 4
8271285.0 15679708 6377225
17 4 4
16484165.0 32553978 9290322
17 4 4
43641599.0 88319860 7115834
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing t

Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering


Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering


Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering


Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering
Processing incoming values
Processing transfering


,catches,transfered,owner,fish
0,2854377.0,0.0,6493928.0,палтус белокорый
1,8846759.0,0.0,5536455.0,палтус белокорый
2,143.0,286.0,9382283.0,палтус белокорый
3,1528.8,2014.0,2680100.0,палтус белокорый
4,43641599.0,88319860.0,7115834.0,палтус белокорый
...,...,...,...,...
167,24333.0,50917.0,3095952.0,камбалы дальневосточные
168,16484165.0,32553978.0,9290322.0,камбалы дальневосточные
169,1528.8,2014.0,2680100.0,камбалы дальневосточные
170,43641599.0,88319860.0,7115834.0,корюшка азиатская зубастая


In [20]:
total_df = pd.read_csv(r'C:\Users\zhiti\Documents\GitHub\Fishing-Control\notebooks\data\unique_owners_stats.csv')
total_df = total_df.drop(axis=1, columns=['Unnamed: 0'])
total_df['owner'] = total_df['owner'].apply(lambda x: int(x))
total_df

,catches,transfered,owner,fish
0,2854377.0,0.0,6493928,палтус белокорый
1,8846759.0,0.0,5536455,палтус белокорый
2,143.0,286.0,9382283,палтус белокорый
3,1528.8,2014.0,2680100,палтус белокорый
4,43641599.0,88319860.0,7115834,палтус белокорый
...,...,...,...,...
167,24333.0,50917.0,3095952,камбалы дальневосточные
168,16484165.0,32553978.0,9290322,камбалы дальневосточные
169,1528.8,2014.0,2680100,камбалы дальневосточные
170,43641599.0,88319860.0,7115834,корюшка азиатская зубастая
